In [ ]:
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [1,073 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:1

In [ ]:
import re
import requests
from bs4 import BeautifulSoup
from pathlib import Path
import pandas as pd
import time
import json
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=chrome_options)

/usr/local/lib/python3.8/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [ ]:
driver.get("https://www.bmwusa.com/build-your-own.html#/series")
driver.maximize_window()

In [ ]:
soup = BeautifulSoup(driver.page_source, 'html')

In [ ]:
containers = soup.find_all('div', {'class':'series-container'})

In [ ]:
columns = ['model_name', 'series_name', 'title', 'starting_msrp', 'shown_msrp', 'img_url']
studio_urls = dict()
cars = pd.DataFrame(columns=columns)

In [ ]:
for container in containers:
  model_name = container.div.text.strip()
  for tile in container.find_all('div', {'class' : 'series-tile'}):
    show_msrp = tile.find('div', {'class':'series-tile__price--short'})
    if show_msrp is not None:
      show_msrp = show_msrp.text.strip()
    else:
      show_msrp = ''
    
    series_name = tile.h2.text.strip()
    studio_urls[series_name] = tile.find('a', {'class' : 'cta-button'})['href']
    rdata = [
        model_name, # model_name
        series_name, # series_name
        tile.p.text.strip(), # title
        tile.find('div', {'class':'series-tile__price'}).text.strip(), # starting_msrp
        show_msrp, # shown_msrp
        tile.img['data-src'] # img_url
    ]
    cars.loc[cars.shape[0]] = rdata

In [ ]:
cars

,model_name,series_name,title,starting_msrp,shown_msrp,img_url
0,iX,iX Sports Activity Vehicle,The all-electric SAV setting standards for aer...,"$84,100","$88,950",https://cache.bmwusa.com/cosy.arox?pov=walkaro...
1,i4,i4 Gran Coupe,Cutting-edge all-electric performance in a lux...,"$51,400","$52,650",https://cache.bmwusa.com/cosy.arox?pov=walkaro...
2,i7,i7 Sedan,The all-electric luxury sedan with executive s...,"$119,300","$119,400",https://cache.bmwusa.com/cosy.arox?pov=walkaro...
3,X1,X1 Sports Activity Vehicle,"The all-new, rugged SAV that is always ready f...","$39,100","$40,850",https://cache.bmwusa.com/cosy.arox?pov=walkaro...
4,X3,X3 Sports Activity Vehicle,The versatile four-door SAV perfect for active...,"$46,200","$46,850",https://cache.bmwusa.com/cosy.arox?pov=walkaro...
5,X4,X4 Sports Activity Coupe,The coupe-inspired adventurer of the X family.,"$54,200","$54,850",https://cache.bmwusa.com/cosy.arox?pov=walkaro...
6,X5,X5 Sports Activity Vehicle,The original that inspired every X to follow.,"$63,900","$75,400",https://cache.bmwusa.com/cosy.arox?pov=walkaro...
7,X6,X6 Sports Activity Coupe,"The refined, reimagined coupe offering of a cl...","$70,100","$74,575",https://cache.bmwusa.com/cosy.arox?pov=walkaro...
8,X7,X7 Sports Activity Vehicle,The largest and most luxurious Sports Activity...,"$77,850",,https://cache.bmwusa.com/cosy.arox?pov=walkaro...
9,X7,ALPINA XB7,The luxury Sports Activity Vehicle with extrao...,"$145,000",,https://cache.bmwusa.com/cosy.arox?pov=walkaro...


In [ ]:
studio_urls

{'iX Sports Activity Vehicle': '#/series/iX/sports-activity-vehicle',
 'i4 Gran Coupe': '#/series/i4/gran-coupe',
 'i7 Sedan': '#/series/i7/sedan',
 'X1 Sports Activity Vehicle': '#/series/X1/sports-activity-vehicle',
 'X3 Sports Activity Vehicle': '#/series/X3/sports-activity-vehicle',
 'X4 Sports Activity Coupe': '#/series/X4/sports-activity-coupe',
 'X5 Sports Activity Vehicle': '#/series/X5/sports-activity-vehicle',
 'X6 Sports Activity Coupe': '#/series/X6/sports-activity-coupe',
 'X7 Sports Activity Vehicle': '#/series/X7/sports-activity-vehicle',
 'ALPINA XB7': '#/series/X7/alpina',
 '2 Series Coupe': '#/series/2/coupe',
 '2 Series Gran Coupe': '#/series/2/gran-coupe',
 '3 Series Sedan': '#/series/3/sedan',
 '4 Series Coupe': '#/series/4/coupe',
 '4 Series Convertible': '#/series/4/convertible',
 '4 Series Gran Coupe': '#/series/4/gran-coupe',
 '5 Series Sedan': '#/series/5/sedan',
 '7 Series Sedan': '#/series/7/sedan',
 '8 Series Coupe': '#/series/8/coupe',
 '8 Series Convertib

In [ ]:
component = pd.DataFrame(columns = ['id', 'type', 'name', 'img_url', 'price', 'series_name'])
interior = pd.DataFrame(columns = ['interiror_ID', 'interior_type', 'upholstery_type'])
exterior = pd.DataFrame(columns = ['exteriror_ID', 'exterior_type', 'color_type', 'wheel_desc', 'wheel_range'])

In [ ]:
for series_name, url in studio_urls.items():
  driver.get("https://www.bmwusa.com/build-your-own.html" + url)
  time.sleep(2)
  driver.maximize_window()
  element = driver.find_element('link text', 'Design')
  element.click()
  time.sleep(2)
  # Color
  url = driver.current_url.replace('/design', '/exterior/color')
  driver.get(url)
  driver.maximize_window()
  time.sleep(2)
  soup = BeautifulSoup(driver.page_source, 'html')
  list_view = soup.find('div', {'class' : 'byo-rail-list-view-items'})
  colours = [i for i in list_view.children if i != ' ' and i != '\n']
  for color_type in colours:
    color_type_name = re.sub(r'\([0-9]*\)', '', color_type.h3.text).strip().lower()
    for option in (i for i in color_type.div.div.contents if i != ' ' and i != '\n'):
      id = component.shape[0]
      if option.figure is None:
        continue
      img_url = re.search(r'(?<=background-image: url\("\s)([\s\S]+)(?=\s"\))', option.figure['style']).group()
      name = option.find('div', {'class':'name'}).text.strip()
      price = option.find('div', {'class':'price'})
      price = price.text.strip() if price is not None else '$0'
      component.loc[component.shape[0]] = [id, 'exterior', name, img_url, price, series_name]
      exterior.loc[exterior.shape[0]] = [id, 'color', color_type_name, None, None]
  
  # Wheels
  url = driver.current_url.replace('/color', '/wheels')
  driver.get(url)
  driver.maximize_window()
  time.sleep(2)
  soup = BeautifulSoup(driver.page_source, 'html')
  list_view = soup.find('div', {'class' : 'flex-container'})
  wheels = [i for i in list_view.children if i != ' ' and i != '\n']
  for wheel in wheels:
    if wheel.figure is None:
      continue
    img_url = re.search(r'(?<=background-image: url\("\s)([\s\S]+)(?=\s"\))', wheel.figure['style']).group()
    name = wheel.find('div', {'class':'name'}).text.strip()
    wheel_desc = wheel.find('div', {'class':'desc'})
    wheel_desc = wheel_desc.text.strip() if wheel_desc is not None else None
    wheel_range = wheel.find('div', {'class':'wheelRange'})
    wheel_range = wheel_range.text.strip() if wheel_range is not None else None
    price = wheel.find('div', {'class':'price'})
    price = price.text.strip() if price is not None else '$0'
    id = component.shape[0]
    component.loc[component.shape[0]] = [id, 'exterior', name, img_url, price, series_name]
    exterior.loc[exterior.shape[0]] = [id, 'wheels', None, wheel_desc, wheel_range]

  # Upholstery
  url = driver.current_url.replace('/exterior/wheels', '/interior/upholstery')
  driver.get(url)
  driver.maximize_window()
  time.sleep(2)
  soup = BeautifulSoup(driver.page_source, 'html')
  list_view = soup.find('div', {'class' : 'byo-rail-list-view-items'})
  upholsteries = [i for i in list_view.children if i != ' ' and i != '\n']
  for upholstery_type in upholsteries:
    upholstery_type_name = re.sub(r'\([0-9]*\)', '', upholstery_type.h3.text).strip().lower()
    for option in upholstery_type.find_all('div', {'class':'byo-rail-option-selector'}):
      id = component.shape[0]
      if option.figure is None:
        continue
      img_url = re.search(r'(?<=background-image: url\("\s)([\s\S]+)(?=\s"\))', option.figure['style']).group()
      name = option.find('div', {'class':'name'}).text.strip()
      price = option.find('div', {'class':'price'})
      price = price.text.strip() if price is not None else '$0'
      component.loc[component.shape[0]] = [id, 'interior', name, img_url, price, series_name]
      interior.loc[interior.shape[0]] = [id, 'upholstery', upholstery_type_name]

  # Trim
  url = driver.current_url.replace('/upholstery', '/trim')
  driver.get(url)
  driver.maximize_window()
  time.sleep(2)
  soup = BeautifulSoup(driver.page_source, 'html')
  trims = soup.find_all('div', {'class' : 'byo-rail-option-selector'})
  for trim in trims:
    if trim.figure is None:
      continue
    img_url = re.search(r'(?<=background-image: url\("\s)([\s\S]+)(?=\s"\))', trim.figure['style']).group()
    name = trim.find('div', {'class':'name'}).text.strip()
    price = trim.find('div', {'class':'price'})
    price = price.text.strip() if price is not None else '$0'
    id = component.shape[0]
    component.loc[component.shape[0]] = [id, 'interior', name, img_url, price, series_name]
    interior.loc[interior.shape[0]] = [id, 'trim', None]

In [ ]:
component['price'] = component['price'].str.extract('(\d+)')
component

,id,type,name,img_url,price,series_name
0,0,exterior,Alpine White,https://cache.bmwusa.com/cosy.arox?brand=wbbi&...,0,iX Sports Activity Vehicle
1,1,exterior,Black Sapphire Metallic,https://cache.bmwusa.com/cosy.arox?brand=wbbi&...,0,iX Sports Activity Vehicle
2,2,exterior,Dark Graphite Metallic,https://cache.bmwusa.com/cosy.arox?brand=wbbi&...,0,iX Sports Activity Vehicle
3,3,exterior,Mineral White Metallic,https://cache.bmwusa.com/cosy.arox?brand=wbbi&...,0,iX Sports Activity Vehicle
4,4,exterior,Phytonic Blue Metallic,https://cache.bmwusa.com/cosy.arox?brand=wbbi&...,0,iX Sports Activity Vehicle
...,...,...,...,...,...,...
1062,1062,interior,Carbon Fiber,https://cache.bmwusa.com/cosy.arox?brand=wbbm&...,0,X6 M Sports Activity Coupe
1063,1063,interior,Aluminum Crossline Interior,https://cache.bmwusa.com/cosy.arox?brand=wbbm&...,0,X6 M Sports Activity Coupe
1064,1064,interior,Individual Piano Black Finish,https://cache.bmwusa.com/cosy.arox?brand=wbbm&...,1,X6 M Sports Activity Coupe
1065,1065,interior,Individual Fine Wood Ash Grain,https://cache.bmwusa.com/cosy.arox?brand=wbbm&...,1,X6 M Sports Activity Coupe


In [ ]:
exterior

,exteriror_ID,exterior_type,color_type,wheel_desc,wheel_range
0,0,color,non-metallic,None,None
1,1,color,metallic,None,None
2,2,color,metallic,None,None
3,3,color,metallic,None,None
4,4,color,metallic,None,None
...,...,...,...,...,...
568,1048,color,metallic,None,None
569,1049,color,metallic,None,None
570,1050,wheels,None,Performance non run-flat,None
571,1051,wheels,None,Performance non run-flat,None


In [ ]:
interior

,interiror_ID,interior_type,upholstery_type
0,19,upholstery,sensatec
1,20,upholstery,sensatec
2,21,upholstery,sensatec
3,22,upholstery,cloth
4,23,upholstery,perforated leather
...,...,...,...
489,1062,trim,None
490,1063,trim,None
491,1064,trim,None
492,1065,trim,None


In [ ]:
component.to_excel('component.xlsx', index=False)
exterior.to_excel('exterior.xlsx', index=False)
interior.to_excel('interior.xlsx', index=False)

from google.colab import files
files.download('component.xlsx')
files.download('exterior.xlsx')
files.download('interior.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from pathlib import Path

In [ ]:
src = Path('temp.html').read_text()

In [ ]:
soup = BeautifulSoup(src, 'html')

In [ ]:
contents = [c for c in soup.find('body').contents if c != '\n' and c != ' ']

In [ ]:
wallpaper_urls = dict()

In [ ]:
for c in contents:
  type_name = c['class'][0].replace('-', ' ').title()
  for url in c.find_all('a'):
    key = cars.loc[(cars.series_name.str.contains('^'+url['title']+'\s(?!M)')) & (cars.series_name.str.contains(type_name)), 'series_name'].to_list()
    if key != []:
      key = key[0]
    else:
      continue
    wallpaper_urls[key] = url['href']

In [ ]:
wallpaper_urls['Z4 Roadster'] = '/vehicles/z4/roadster/overview.html'

In [ ]:
wallpaper_urls

{'X1 Sports Activity Vehicle': '/vehicles/x-models/x1/sports-activity-vehicle/overview.html',
 'X3 Sports Activity Vehicle': '/vehicles/x-models/x3/sports-activity-vehicle/overview.html',
 'X4 Sports Activity Coupe': '/vehicles/x-models/x4/sports-activity-coupe/overview.html',
 'X5 Sports Activity Vehicle': '/vehicles/x-models/x5/sports-activity-vehicle/overview.html',
 'X6 Sports Activity Coupe': '/vehicles/x-models/x6/sports-activity-coupe/overview.html',
 'X7 Sports Activity Vehicle': '/vehicles/x-models/x7/sports-activity-vehicle/overview.html',
 'iX Sports Activity Vehicle': '/vehicles/all-electric/ix/sports-activity-vehicle/overview.html',
 '2 Series Gran Coupe': '/vehicles/2-series/gran-coupe/overview.html',
 '4 Series Gran Coupe': '/vehicles/4-series/gran-coupe/overview.html',
 '8 Series Gran Coupe': '/vehicles/8-series/gran-coupe/overview.html',
 'i4 Gran Coupe': '/vehicles/all-electric/i4/gran-coupe/overview.html',
 '3 Series Sedan': '/vehicles/3-series/sedan/overview.html',


In [ ]:
cars2 = cars.copy()
cars2['wallpaper'] = None

In [ ]:
faq = pd.DataFrame(columns=['series_name', 'question', 'answer'])

In [ ]:
for series_name, url in wallpaper_urls.items():
  driver.get('https://www.bmwusa.com' + url)
  driver.maximize_window()
  soup = BeautifulSoup(driver.page_source, 'html')
  cars2.loc[cars2.series_name == series_name, 'wallpaper'] = 'www.bmwusa.com' + soup.picture.source['data-srcset']
  for q in soup.find_all('div', {'class' : 'faq'}):
    if q != ' ':
      question = q.find('div', {'class' : 'section-header'})
      if question is None:
        continue
      else:
        rdata = [
          series_name,
          question.text.strip(),
          q.find('div', {'class' : 'faq-response'}).text.strip()
        ]
        faq.loc[faq.shape[0]] = rdata

In [ ]:
faq.answer = faq.answer.str.replace(r'\s*\n+\s*', r'\n', regex=True).str.replace(r'\n\$', ' $', regex=True)

In [ ]:
faq

,series_name,question,answer
0,X1 Sports Activity Vehicle,How many seats does the X1 have?,The 2023 BMW X1 is a four-door Sports Activity...
1,X1 Sports Activity Vehicle,What additional options and packages are avail...,The 2023 BMW X1 SAV has an expansive range of ...
2,X1 Sports Activity Vehicle,Is the BMW X1 a reliable and safe family car?,The 2023 BMW X1 comes standard with Active Dri...
3,X1 Sports Activity Vehicle,Does the BMW X1 have all-wheel drive?,"Yes. Every 2023 BMW X1 comes with xDrive, BMW’..."
4,X1 Sports Activity Vehicle,What smart features are included in the BMW X1?,This intelligent SAV includes the all-new BMW ...
...,...,...,...
275,Z4 Roadster,How many seats does the BMW Z4 Roadster have?,The 2023 BMW Z4 roadster convertible has seati...
276,Z4 Roadster,What smart features does the Z4 offer?,The BMW Z4 is equipped with an advanced operat...
277,Z4 Roadster,Statistics Summary,"Dimensions: From 170.7"" L x 73.4"" W x 51.5"" H\..."
278,Z4 Roadster,Engine & Drivetrain Variants,2.0-liter BMW TwinPower Turbo inline 4-cylinde...


In [ ]:
cars2

,model_name,series_name,title,starting_msrp,shown_msrp,img_url,wallpaper
0,iX,iX Sports Activity Vehicle,The all-electric SAV setting standards for aer...,"$84,100","$88,950",https://cache.bmwusa.com/cosy.arox?pov=walkaro...,www.bmwusa.com/content/dam/bmwusa/common/vehic...
1,i4,i4 Gran Coupe,Cutting-edge all-electric performance in a lux...,"$51,400","$52,650",https://cache.bmwusa.com/cosy.arox?pov=walkaro...,www.bmwusa.com/content/dam/bmwusa/common/vehic...
2,i7,i7 Sedan,The all-electric luxury sedan with executive s...,"$119,300","$119,400",https://cache.bmwusa.com/cosy.arox?pov=walkaro...,www.bmwusa.com/content/dam/bmwusa/common/vehic...
3,X1,X1 Sports Activity Vehicle,"The all-new, rugged SAV that is always ready f...","$39,100","$40,850",https://cache.bmwusa.com/cosy.arox?pov=walkaro...,www.bmwusa.com/content/dam/bmwusa/common/vehic...
4,X3,X3 Sports Activity Vehicle,The versatile four-door SAV perfect for active...,"$46,200","$46,850",https://cache.bmwusa.com/cosy.arox?pov=walkaro...,www.bmwusa.com/content/dam/bmwusa/common/vehic...
5,X4,X4 Sports Activity Coupe,The coupe-inspired adventurer of the X family.,"$54,200","$54,850",https://cache.bmwusa.com/cosy.arox?pov=walkaro...,www.bmwusa.com/content/dam/bmwusa/common/vehic...
6,X5,X5 Sports Activity Vehicle,The original that inspired every X to follow.,"$63,900","$75,400",https://cache.bmwusa.com/cosy.arox?pov=walkaro...,www.bmwusa.com/content/dam/bmwusa/common/vehic...
7,X6,X6 Sports Activity Coupe,"The refined, reimagined coupe offering of a cl...","$70,100","$74,575",https://cache.bmwusa.com/cosy.arox?pov=walkaro...,www.bmwusa.com/content/dam/bmwusa/common/vehic...
8,X7,X7 Sports Activity Vehicle,The largest and most luxurious Sports Activity...,"$77,850",,https://cache.bmwusa.com/cosy.arox?pov=walkaro...,www.bmwusa.com/content/dam/bmwusa/common/vehic...
9,X7,ALPINA XB7,The luxury Sports Activity Vehicle with extrao...,"$145,000",,https://cache.bmwusa.com/cosy.arox?pov=walkaro...,None


In [ ]:
faq.to_excel('faq.xlsx', index=False)

In [ ]:
cars2.to_excel('cars.xlsx', index=False)

In [ ]:
from google.colab import files

In [ ]:
files.download('cars.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('faq.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>